# Spam and Ham

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import tarfile

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100%; }</style>"))
np.set_printoptions(precision=3, suppress=True)

In [2]:
DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham.tar.bz2"
SPAM_URL = DOWNLOAD_ROOT + "20030228_spam.tar.bz2"
EMAILS_PATH = Path.cwd() / "datasets" / "spamham"

def fetch_spam_data(ham_url=HAM_URL, spam_url=SPAM_URL, emails_path=EMAILS_PATH):
    emails_path.mkdir(parents=True, exist_ok=True)
    for filename, url in [("ham.tar.bz2", ham_url), ("spam.tar.bz2", spam_url)]:
        resp = requests.get(url)
        local_file = emails_path / filename
        with open(local_file, "wb") as f:
            f.write(resp.content)
        file_bz2 = tarfile.open(local_file)
        file_bz2.extractall(path=emails_path)
        file_bz2.close()

In [3]:
fetch_spam_data()

In [4]:
ham_dir = EMAILS_PATH / "easy_ham"
spam_dir = EMAILS_PATH / "spam"
    
ham_files = list(ham_dir.glob("*.*")) 
spam_files = list(spam_dir.glob("*.*"))

ham_num = len(ham_files)
spam_num = len(spam_files)

print(ham_num, spam_num)

2500 500


In [5]:
from email.parser import BytesParser
from email.policy import default

def load_email(filename):
    with open(filename, "rb") as f:
        return BytesParser(policy=default).parse(f)

In [6]:
ham_emails = [load_email(file) for file in ham_files]
spam_emails = [load_email(file) for file in spam_files]

In [7]:
ham_idx = list()
spam_idx = list()
ham_set = set()
spam_set = set()

for idx, email in enumerate(ham_emails):
    ham_set.add(type(email.get_payload()))
    if isinstance(email.get_payload(), list):
        ham_idx.append(idx)

for idx, email in enumerate(spam_emails):
    spam_set.add(type(email.get_payload()))
    if isinstance(email.get_payload(), list):
        spam_idx.append(idx)
    
print("ham", ham_idx, ham_set, "spam", spam_idx, spam_set, sep="\n")

ham
[13, 61, 62, 66, 69, 165, 367, 385, 386, 387, 388, 577, 703, 774, 848, 882, 943, 944, 946, 947, 948, 951, 960, 962, 963, 965, 967, 974, 985, 987, 992, 996, 1002, 1028, 1044, 1052, 1064, 1066, 1068, 1093, 1095, 1126, 1128, 1129, 1130, 1131, 1135, 1136, 1147, 1148, 1151, 1158, 1161, 1167, 1169, 1180, 1182, 1215, 1224, 1232, 1293, 1313, 1335, 1352, 1357, 1391, 1396, 1402, 1405, 1424, 1435, 1445, 1447, 1467, 1468, 1472, 1473, 1475, 1490, 1508, 1541, 1557, 1560, 1564, 1566, 1569, 1570, 1590, 1604, 1608, 1609, 1622]
{<class 'list'>, <class 'str'>}
spam
[21, 22, 23, 37, 38, 73, 77, 79, 81, 86, 89, 91, 94, 95, 104, 114, 119, 123, 125, 127, 134, 136, 149, 179, 185, 186, 190, 191, 193, 197, 202, 205, 206, 207, 208, 213, 215, 218, 222, 231, 236, 237, 238, 240, 247, 251, 255, 259, 260, 261, 263, 264, 268, 270, 272, 274, 282, 286, 290, 292, 297, 298, 302, 306, 307, 308, 310, 314, 329, 331, 332, 333, 334, 337, 338, 339, 340, 343, 351, 364, 365, 369, 383, 388, 389, 398, 402, 420, 434, 435, 439, 4

### HAM - Regular String

In [8]:
print(type(ham_emails[0]))
print(ham_emails[0].get_content_type())
print(ham_emails[0].get_content().strip())
print(ham_emails[0].get_payload())

<class 'email.message.EmailMessage'>
text/plain
Date:        Wed, 21 Aug 2002 10:54:46 -0500
    From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>
    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>


  | I can't reproduce this error.

For me it is very repeatable... (like every time, without fail).

This is the debug log of the pick happening ...

18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}
18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury
18:19:04 Ftoc_PickMsgs {{1 hit}}
18:19:04 Marking 1 hits
18:19:04 tkerror: syntax error in expression "int ...

Note, if I run the pick command by hand ...

delta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury
1 hit

That's where the "1 hit" comes from (obviously).  The version of nmh I'm
using is ...

delta$ pick -version
pick -- nmh-1.0.4 [compile

### HAM - Multipart

In [9]:
print(ham_emails[13].get_content_type())
multipart_email = ham_emails[13].get_payload()
print(multipart_email)

for part in multipart_email:
    print(part.get_payload())

multipart/signed
[<email.message.EmailMessage object at 0x10adbee50>, <email.message.EmailMessage object at 0x117ca4bd0>]
> From:  Chris Garrigues <cwg-exmh@DeepEddy.Com>
> Date:  Wed, 21 Aug 2002 10:40:39 -0500
>
> > From:  Chris Garrigues <cwg-exmh@DeepEddy.Com>
> > Date:  Wed, 21 Aug 2002 10:17:45 -0500
> >
> > Ouch...I'll get right on it.
> > 
> > > From:  Robert Elz <kre@munnari.OZ.AU>
> > > Date:  Wed, 21 Aug 2002 19:30:01 +0700
> > >
> > > Any chance of having that lengthen instead?   I like all my exmh stuff
> > > in nice columns (fits the display better).   That is, I use the detache
> d
> > > folder list, one column.   The main exmh window takes up full screen,
> > > top to bottom, but less than half the width, etc...
> 
> I thought about that.  The first order approximation would be to just add 
> using pack .... -side top instead of pack ... -side left, however, since their 
> each a different width, it would look funny.

I've done this.  It's not as pretty as I think it sh

### SPAM - Regular String

In [10]:
print(spam_emails[9].get_content_type())
print(spam_emails[9].get_content().strip())
print(spam_emails[9].get_payload())

text/html
Dear ricardo1 ,

<html>
<body>
<center>
<b><font color = "red" size = "+2.5">COST EFFECTIVE Direct Email Advertising</font><br>
<font color = "blue" size = "+2">Promote Your Business For As Low As </font><br>
<font color = "red" size = "+2">$50</font> <font color = "blue" size = "+2">Per 
<font color = "red" size = "+2">1 Million</font>
<font color = "blue" size = "+2"> Email Addresses</font></font><p>
<b><font color = "#44C300" size ="+2">MAXIMIZE YOUR MARKETING DOLLARS!<p></FONT></b>
<font size = "+2">Complete and fax this information form to 309-407-7378.<Br>
A Consultant will contact you to discuss your marketing needs.<br>
</font></font>
<Table><tr><td>
<font size = "+1"><b>NAME:___________________________________________________________________<br>
<font size = "+1"><b>COMPANY:_______________________________________________________________<br>
<font size = "+1"><b>ADDRESS:________________________________________________________________<br>
<font size = "+1"><b>CITY:____

### SPAM - Multipart

In [11]:
print(spam_emails[21].get_content_type())

multipart_email = spam_emails[21].get_payload()
print(multipart_email)

for part in multipart_email:
    print(part.get_payload())

multipart/mixed
[<email.message.EmailMessage object at 0x11b47a650>, <email.message.EmailMessage object at 0x11b474a10>]
URGENT PRIVATE & EXTREMELY CONFIDENTIAL



Dear =2C

With profound interest and in utmost confidence=2C I am
soliciting your immediate assistance or co-operation
as to enable us round up an opportunity within my
capability as a result of the death of one of our
contractor =28Beneficiary=29=2E You should not be surprised
as to how I got your contact=2C you were highly
recommended to me with the believe that you are
competent=2C reliable=2C Trustworthy and confident=2E

I am  Dr=2E Bello Ahmed=2C Chief Auditor=2C Special Project
and Foreign Contract Regularization and Disbursement=2C
in the Office of the Auditor General of the Federation
of Federal Republic of Nigeria=2E We work in hand with
the Senate Committee on Foreign Contract Payment=2E Our
duty is to ensure that all contractors are paid their
contract sum in due time=2E

This last payment quarter=2C a total of 3

### Functions to Get Email Structures and Count Them

In [12]:
def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()
    if isinstance(payload, list):
        return f"multipart({(', '.join(get_email_structure(sub_email) for sub_email in payload))})"
    else:
        return email.get_content_type()

In [13]:
def count_email_structures(emails):
    structures = dict({})
    for email in emails:
        structure = get_email_structure(email)
        if structure in structures.keys():
            structures[structure] += 1
        else:
            structures[structure] = 1
    return structures

In [14]:
for email_type, emails in [("HAM", ham_emails), ("SPAM", spam_emails)]:
    print(email_type)
    structures = count_email_structures(emails)
    for structure, structure_cnt in sorted(structures.items(), key=lambda kv: (kv[1], kv[0]), reverse=True):
        print(structure, structure_cnt)
    print("\n")

HAM
text/plain 2408
multipart(text/plain, application/pgp-signature) 66
multipart(text/plain, text/html) 8
multipart(text/plain, text/plain) 4
multipart(text/plain) 3
multipart(text/plain, application/octet-stream) 2
multipart(text/plain, video/mng) 1
multipart(text/plain, text/enriched) 1
multipart(text/plain, multipart(text/plain, text/plain), text/rfc822-headers) 1
multipart(text/plain, multipart(text/plain, text/plain), multipart(multipart(text/plain, application/x-pkcs7-signature))) 1
multipart(text/plain, multipart(text/plain)) 1
multipart(text/plain, application/x-pkcs7-signature) 1
multipart(text/plain, application/x-java-applet) 1
multipart(text/plain, application/ms-tnef, text/plain) 1
multipart(multipart(text/plain, text/plain, text/plain), application/pgp-signature) 1


SPAM
text/plain 218
text/html 183
multipart(text/plain, text/html) 45
multipart(text/html) 20
multipart(text/plain) 19
multipart(multipart(text/html)) 5
multipart(text/plain, image/jpeg) 3
multipart(text/htm

In [15]:
print(spam_emails[0].items(), spam_emails[0].keys(), spam_emails[0].values(), sep="\n")

[('Return-Path', '<12a1mailbot1@web.de>'), ('Delivered-To', 'zzzz@localhost.spamassassin.taint.org'), ('Received', 'from localhost (localhost [127.0.0.1])\tby phobos.labs.spamassassin.taint.org (Postfix) with ESMTP id 136B943C32\tfor <zzzz@localhost>; Thu, 22 Aug 2002 08:17:21 -0400 (EDT)'), ('Received', 'from mail.webnote.net [193.120.211.219]\tby localhost with POP3 (fetchmail-5.9.0)\tfor zzzz@localhost (single-drop); Thu, 22 Aug 2002 13:17:21 +0100 (IST)'), ('Received', 'from dd_it7 ([210.97.77.167])\tby webnote.net (8.9.3/8.9.3) with ESMTP id NAA04623\tfor <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 13:09:41 +0100'), ('From', '12a1mailbot1@web.de'), ('Received', 'from r-smtp.korea.com - 203.122.2.197 by dd_it7  with Microsoft SMTPSVC(5.5.1775.675.6);\t Sat, 24 Aug 2002 09:42:10 +0900'), ('To', 'dcek1a1@netsgo.com'), ('Subject', 'Life Insurance - Why Pay More?'), ('Date', 'Wed, 21 Aug 2002 20:31:57 -1600'), ('MIME-Version', '1.0'), ('Message-ID', '<0103c1042001882DD_IT7@dd_it7>'

In [16]:
for header, value in spam_emails[0].items():
    print(header, ": ", value)

Return-Path :  <12a1mailbot1@web.de>
Delivered-To :  zzzz@localhost.spamassassin.taint.org
Received :  from localhost (localhost [127.0.0.1])	by phobos.labs.spamassassin.taint.org (Postfix) with ESMTP id 136B943C32	for <zzzz@localhost>; Thu, 22 Aug 2002 08:17:21 -0400 (EDT)
Received :  from mail.webnote.net [193.120.211.219]	by localhost with POP3 (fetchmail-5.9.0)	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 13:17:21 +0100 (IST)
Received :  from dd_it7 ([210.97.77.167])	by webnote.net (8.9.3/8.9.3) with ESMTP id NAA04623	for <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 13:09:41 +0100
From :  12a1mailbot1@web.de
Received :  from r-smtp.korea.com - 203.122.2.197 by dd_it7  with Microsoft SMTPSVC(5.5.1775.675.6);	 Sat, 24 Aug 2002 09:42:10 +0900
To :  dcek1a1@netsgo.com
Subject :  Life Insurance - Why Pay More?
Date :  Wed, 21 Aug 2002 20:31:57 -1600
MIME-Version :  1.0
Message-ID :  <0103c1042001882DD_IT7@dd_it7>
Content-Type :  text/html; charset="iso-8859-1"
Content-Transfer-

In [17]:
spam_emails[0]["Subject"]

'Life Insurance - Why Pay More?'

### Function to remove HTML tags and convert to plain text

In [18]:
import re
from html import unescape

def html_to_plain_text(html):
    # multi-line, dot matches all (including newline), ignore case 
    # re.S: Make the '.' special character match any character at all, including a newline; without this flag, '.' will match anything except a newline.
    # re.I: Ignore case
    # <head.*?> --> ? is non-greedy, matches up to as few characters possible
    text = re.sub("<head.*?>.*?</head>", "", html, flags=re.M | re.S | re.I)
    text = re.sub("<a\s.*?>", " HYPERLINK ", text, flags=re.M | re.S | re.I)
    text = re.sub("<.*?>", "", text, flags=re.M | re.S)
    text = re.sub(r"(\s*\n)+", "\n", text, flags=re.M | re.S)
    return unescape(text)

### Prepare Training and Test Data

In [19]:
from sklearn.model_selection import train_test_split

X = np.array(ham_emails + spam_emails)
y = np.array([0] * ham_num + [1] * spam_num)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
html_spam_emails = [email for email in X_train[y_train==1] if get_email_structure(email) == "text/html"]
sample_email = html_spam_emails[7]
print(sample_email.get_content().strip()[:1000])

<HTML><HEAD><TITLE></TITLE><META http-equiv="Content-Type" content="text/html; charset=windows-1252"><STYLE>A:link {TEX-DECORATION: none}A:active {TEXT-DECORATION: none}A:visited {TEXT-DECORATION: none}A:hover {COLOR: #0033ff; TEXT-DECORATION: underline}</STYLE><META content="MSHTML 6.00.2713.1100" name="GENERATOR"></HEAD>
<BODY text="#000000" vLink="#0033ff" link="#0033ff" bgColor="#CCCC99"><TABLE borderColor="#660000" cellSpacing="0" cellPadding="0" border="0" width="100%"><TR><TD bgColor="#CCCC99" valign="top" colspan="2" height="27">
<font size="6" face="Arial, Helvetica, sans-serif" color="#660000">
<b>OTC</b></font></TD></TR><TR><TD height="2" bgcolor="#6a694f">
<font size="5" face="Times New Roman, Times, serif" color="#FFFFFF">
<b>&nbsp;Newsletter</b></font></TD><TD height="2" bgcolor="#6a694f"><div align="right"><font color="#FFFFFF">
<b>Discover Tomorrow's Winners&nbsp;</b></font></div></TD></TR><TR><TD height="25" colspan="2" bgcolor="#CCCC99"><table width="100%" border="0" 

In [21]:
print(html_to_plain_text(sample_email.get_content())[:1000])


OTC
 Newsletter
Discover Tomorrow's Winners 
For Immediate Release
Cal-Bay (Stock Symbol: CBYI)
Watch for analyst "Strong Buy Recommendations" and several advisory newsletters picking CBYI.  CBYI has filed to be traded on the OTCBB, share prices historically INCREASE when companies get listed on this larger trading exchange. CBYI is trading around 25 cents and should skyrocket to $2.66 - $3.25 a share in the near future.
Put CBYI on your watch list, acquire a position TODAY.
REASONS TO INVEST IN CBYI
A profitable company and is on track to beat ALL earnings estimates!
One of the FASTEST growing distributors in environmental & safety equipment instruments.
Excellent management team, several EXCLUSIVE contracts.  IMPRESSIVE client list including the U.S. Air Force, Anheuser-Busch, Chevron Refining and Mitsubishi Heavy Industries, GE-Energy & Environmental Research.
RAPIDLY GROWING INDUSTRY
Industry revenues exceed $900 million, estimates indicate that there could be as much as $25 billi

### Function to Handle Different Text-Based Email Types (text/plain and text/html)

In [22]:
def email_to_text(email):
    html = None
    for part in email.walk():
        ctype = part.get_content_type()
        if ctype not in ("text/plain", "text/html"):
            continue
        try:
            content = part.get_content()
        except:
            content = str(part.get_payload())
        if ctype == "text/plain":
            return content
        else:
            html = content
            return html_to_plain_text(html)

In [23]:
email_to_text(sample_email)

'\nOTC\n\xa0Newsletter\nDiscover Tomorrow\'s Winners\xa0\nFor Immediate Release\nCal-Bay (Stock Symbol: CBYI)\nWatch for analyst "Strong Buy Recommendations" and several advisory newsletters picking CBYI.  CBYI has filed to be traded on the OTCBB, share prices historically INCREASE when companies get listed on this larger trading exchange. CBYI is trading around 25 cents and should skyrocket to $2.66 - $3.25 a share in the near future.\nPut CBYI on your watch list, acquire a position TODAY.\nREASONS TO INVEST IN CBYI\nA profitable company and is on track to beat ALL earnings estimates!\nOne of the FASTEST growing distributors in environmental & safety equipment instruments.\nExcellent management team, several EXCLUSIVE contracts.  IMPRESSIVE client list including the U.S. Air Force, Anheuser-Busch, Chevron Refining and Mitsubishi Heavy Industries, GE-Energy & Environmental Research.\nRAPIDLY GROWING INDUSTRY\nIndustry revenues exceed $900 million, estimates indicate that there could be

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

try:
    import nltk
    stemmer = nltk.PorterStemmer()
    for word in ("Computations", "Computation", "Computing", "Computed", "Compute", "Compulsive"):
        print(stemmer.stem(word))
except ImportError:
    print("Error: stemming requires the NLTK module.")
    stemmer = None

comput
comput
comput
comput
comput
compuls


In [25]:
try:
    import urlextract
    
    url_extractor = urlextract.URLExtract()
    print(url_extractor.find_urls("Will it detect github.com and https://youtu.be/7Pq-S557XQU?t=3m32s"))
except ImportError:
    print("Error: replacing URLs requires the urlextract module.")
    url_extractor = None

['github.com', 'https://youtu.be/7Pq-S557XQU?t=3m32s']


### Transformer to Convert Email to Word Counts

In [26]:
from collections import Counter
from sklearn.base import BaseEstimator, TransformerMixin

class EmailtoWordCounterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, strip_headers=True, lower_case=True, remove_punctuations=True, replace_urls=True, replace_numbers=True, stemming=True):
        self.strip_headers = strip_headers
        self.lower_case = lower_case
        self.remove_punctuations = remove_punctuations
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.stemming = stemming
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_transformed = list()
        for email in X:
            text = email_to_text(email) or ""
            if self.lower_case:
                text = text.lower()
            if self.replace_urls and url_extractor is not None:
                urls = list(set(url_extractor.find_urls(text)))
                urls.sort(key=lambda url: len(url), reverse=True)
                for url in urls:
                    text = text.replace(url, " URL ")
            if self.replace_numbers:
                text = re.sub(r"\d+(?:\.\d*(?:[eE]d+))?", " NUMBER", text)
            if self.remove_punctuations:
                text = re.sub(r"\W+", " ", text, flags=re.M)
            word_counts = Counter(text.split())
            if self.stemming and stemmer:
                stemmed_word_counts = Counter()
                for word, count in word_counts.items():
                    stemmed_word = stemmer.stem(word)
                    stemmed_word_counts[stemmed_word] += count
                word_counts = stemmed_word_counts
            X_transformed.append(word_counts)
        return np.array(X_transformed)                

In [27]:
X_sample = X_train[:3]
X_sample_word_counts = EmailtoWordCounterTransformer().fit_transform(X_sample)
X_sample_word_counts

array([Counter({'chuck': 1, 'murcko': 1, 'wrote': 1, 'stuff': 1, 'yawn': 1, 'r': 1}),
       Counter({'the': 11, 'of': 9, 'and': 8, 'all': 3, 'christian': 3, 'to': 3, 'by': 3, 'jefferson': 2, 'i': 2, 'have': 2, 'superstit': 2, 'one': 2, 'on': 2, 'been': 2, 'ha': 2, 'half': 2, 'rogueri': 2, 'teach': 2, 'jesu': 2, 'some': 1, 'interest': 1, 'quot': 1, 'url': 1, 'thoma': 1, 'examin': 1, 'known': 1, 'word': 1, 'do': 1, 'not': 1, 'find': 1, 'in': 1, 'our': 1, 'particular': 1, 'redeem': 1, 'featur': 1, 'they': 1, 'are': 1, 'alik': 1, 'found': 1, 'fabl': 1, 'mytholog': 1, 'million': 1, 'innoc': 1, 'men': 1, 'women': 1, 'children': 1, 'sinc': 1, 'introduct': 1, 'burnt': 1, 'tortur': 1, 'fine': 1, 'imprison': 1, 'what': 1, 'effect': 1, 'thi': 1, 'coercion': 1, 'make': 1, 'world': 1, 'fool': 1, 'other': 1, 'hypocrit': 1, 'support': 1, 'error': 1, 'over': 1, 'earth': 1, 'six': 1, 'histor': 1, 'american': 1, 'john': 1, 'e': 1, 'remsburg': 1, 'letter': 1, 'william': 1, 'short': 1, 'again': 1, 'becom

In [28]:
X_sample_word_counts[0].most_common(10)

[('chuck', 1),
 ('murcko', 1),
 ('wrote', 1),
 ('stuff', 1),
 ('yawn', 1),
 ('r', 1)]

In [29]:
for elem in X_sample_word_counts[0].elements():
    print(elem)

chuck
murcko
wrote
stuff
yawn
r


### Transformer to Convert Word Counts into Sparse Matrix

In [30]:
from scipy.sparse import csr_matrix
# class scipy.sparse.csr_matrix(arg1, shape=None, dtype=None, copy=False)[source]
# Compressed Sparse Row matrix

class WordCountToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10)
        self.most_common_ = total_count.most_common()[: self.vocabulary_size]
        self.vocabulary_ = {word: index + 1 for index, (word, count) in enumerate(self.most_common_)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size + 1))

In [31]:
vocab_transformer = WordCountToVectorTransformer(vocabulary_size=10)
X_sample_vector = vocab_transformer.fit_transform(X_sample_word_counts)
X_sample_vector

<3x11 sparse matrix of type '<class 'numpy.longlong'>'
	with 20 stored elements in Compressed Sparse Row format>

In [32]:
X_sample_vector.toarray()

array([[ 6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [99, 11,  9,  8,  3,  1,  3,  1,  3,  2,  3],
       [67,  0,  1,  2,  3,  4,  1,  2,  0,  1,  0]], dtype=int64)

In [33]:
vocab_transformer.vocabulary_

{'the': 1,
 'of': 2,
 'and': 3,
 'to': 4,
 'url': 5,
 'all': 6,
 'in': 7,
 'christian': 8,
 'on': 9,
 'by': 10}

### Full Pipeline to Convert from Email to Sparse Matrix

In [34]:
from sklearn.pipeline import Pipeline

preprocessing_pipeline = Pipeline([
    ("email_to_wordcount", EmailtoWordCounterTransformer()),
    ("wordcount_to_vector", WordCountToVectorTransformer())
])

X_train_transformed = preprocessing_pipeline.fit_transform(X_train)

### Apply Classification Model to Training and Test Data

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# class sklearn.linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, 
#                                               solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
# solver{‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’}, default=’lbfgs’
# Algorithm to use in the optimization problem.
# For small datasets, ‘liblinear’ is a good choice, whereas ‘sag’ and ‘saga’ are faster for large ones.
# For multiclass problems, only ‘newton-cg’, ‘sag’, ‘saga’ and ‘lbfgs’ handle multinomial loss; ‘liblinear’ is limited to one-versus-rest schemes.
# ‘newton-cg’, ‘lbfgs’, ‘sag’ and ‘saga’ handle L2 or no penalty
# ‘liblinear’ and ‘saga’ also handle L1 penalty
# ‘saga’ also supports ‘elasticnet’ penalty
# ‘liblinear’ does not support setting penalty='none'

log_clf = LogisticRegression(solver="liblinear", random_state=42)
score = cross_val_score(log_clf, X_train_transformed, y_train, cv=10, verbose=3, n_jobs=-1)
score.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    2.3s remaining:    5.4s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    2.4s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.5s finished


0.9875

In [36]:
from sklearn.metrics import precision_score, recall_score

X_test_transformed = preprocessing_pipeline.transform(X_test)

log_clf = LogisticRegression(solver="liblinear", random_state=42)
log_clf.fit(X_train_transformed, y_train)

y_pred = log_clf.predict(X_test_transformed)

print(f"Precision: {100 * precision_score(y_test, y_pred):.2f}%")
print(f"Recall: {100 * recall_score(y_test, y_pred):.2f}%")

Precision: 94.90%
Recall: 97.89%


In [37]:
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import cross_val_predict

y_scores = cross_val_predict(log_clf, X_train_transformed, y_train, cv=10, method="predict_proba", verbose=3, n_jobs=-1)
y_scores[:10]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    1.7s remaining:    3.9s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    1.7s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.0s finished


array([[0.793, 0.207],
       [0.961, 0.039],
       [1.   , 0.   ],
       [0.986, 0.014],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [0.125, 0.875],
       [1.   , 0.   ]])

In [38]:
precisions, recalls, thresholds = precision_recall_curve(y_train, y_scores[:, 1])

In [39]:
precisions

array([0.169, 0.169, 0.169, ..., 0.938, 0.933, 1.   ])

In [40]:
recalls

array([1.   , 0.998, 0.998, ..., 0.037, 0.035, 0.   ])